In [1]:
import datetime
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import GridSearchCV

In [2]:
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
import datetime
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import GridSearchCV
import numpy as np
from scipy.stats import randint
from catboost import CatBoostClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt
from sklearn.metrics import f1_score, make_scorer

f1 = make_scorer(f1_score , average='micro')

#  {'depth': 5, 'iterations': 500, 'learning_rate': 0.01}
print(" CATBOOST ".center(80, "="))

print("Loading dataset...")

df = pd.read_csv("/Users/pryda/Documents/Projets_Jedha/2_ConversioNRate/conversion_data_train.csv")

X = df.drop(["converted"], axis=1)
y = df["converted"]

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

categoricals = ["country", "new_user", "source"]

# model = CatBoostClassifier(iterations=500, learning_rate=0.01, depth=5, loss_function="Logloss", eval_metric="F1", cat_features=categoricals, verbose=100) # poop

# model = CatBoostClassifier(loss_function="Logloss", eval_metric="F1", cat_features=categoricals, verbose=100) # 0.777 train, 0.7517 test

# model = CatBoostClassifier(loss_function="Logloss", eval_metric="F1", cat_features=categoricals, verbose=100, depth=9, learning_rate=0.2, rsm=1) # f1-score on train set :  0.7910958904109588
# f1-score on test set :  0.7397585592717197

cbc = CatBoostClassifier(cat_features=categoricals, eval_metric="F1", verbose=100)
param_dist = {
    # "learning_rate": np.linspace(0,0.2,5),
    # "depth": randint(3, 10),
    # "iterations": randint(100, 500),
    "rsm": np.linspace(0,1),
    "learning_rate": sp_randFloat(0.01, 0.8),
    "depth": sp_randInt(6, 10),
    "iterations": sp_randInt(10, 1000),
    "border_count": sp_randInt(1, 255),
    'l2_leaf_reg': sp_randInt(1, 10),
    'loss_function': ['Logloss', 'CrossEntropy'],
}

model = RandomizedSearchCV(cbc, param_distributions=param_dist, n_iter=10, cv=10, scoring=f1, n_jobs=-1)

model.fit(X_train, Y_train, eval_set=(X_test, Y_test))

Y_train_pred = model.predict(X_train)
Y_test_pred = model.predict(X_test)

# Print the tuned parameters and score
print(model.best_params_)
print(model.best_score_)

print("f1-score on train set : ", f1_score(Y_train, Y_train_pred))
print("f1-score on test set : ", f1_score(Y_test, Y_test_pred))

# model.save_model("catboost_v1_randomsearch")

=================================== CATBOOST ===================================
Loading dataset...
0:	learn: 0.7090158	test: 0.6997549	best: 0.6997549 (0)	total: 96.2ms	remaining: 1m 13s
100:	learn: 0.7689878	test: 0.7490231	best: 0.7513514 (62)	total: 3.01s	remaining: 20s
200:	learn: 0.7703292	test: 0.7483444	best: 0.7513514 (62)	total: 6.14s	remaining: 17.4s
300:	learn: 0.7721063	test: 0.7493245	best: 0.7513514 (62)	total: 9.49s	remaining: 14.8s
400:	learn: 0.7737961	test: 0.7500000	best: 0.7513514 (62)	total: 12.6s	remaining: 11.6s
500:	learn: 0.7759916	test: 0.7487981	best: 0.7513514 (62)	total: 15.8s	remaining: 8.5s
600:	learn: 0.7790942	test: 0.7494762	best: 0.7513514 (62)	total: 19s	remaining: 5.34s
700:	learn: 0.7791876	test: 0.7483523	best: 0.7513514 (62)	total: 22.4s	remaining: 2.21s
769:	learn: 0.7807222	test: 0.7488776	best: 0.7513514 (62)	total: 24.7s	remaining: 0us

bestTest = 0.7513513514
bestIteration = 62

Shrink model to first 63 iterations.
{'border_count': 188, 'de

In [ ]:
# Concatenate our train and test set to train your best classifier on all data with labels
X = np.append(X_train,X_test,axis=0)
Y = np.append(Y_train,Y_test)

model.fit(X,Y)

In [ ]:
df_test = pd.read_csv("/Users/pryda/Documents/Projets_Jedha/2_ConversioNRate/conversion_data_test.csv")

predictions = model.predict(df_test)
print(predictions)

pred_df = pd.DataFrame(columns=['converted'], data=predictions)
# pred_df = pd.DataFrame({ "converted": np.array(predictions)[:, -1] })
pred_df.to_csv(f"conversion_data_test_predictions_LV_catboost_split{datetime.datetime.now().timestamp()}.csv",
               index=False)